Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

## ONNX Runtime Sentiment Analysis with MobileBert model


In this tutorial, you will learn the end-to-end steps from obtaining a HuggingFace model, converting to ONNX format, adding pre/post processing steps to the ONNX model using onnxruntime-extensions library and finally plug in and apply inference in a sample mobile android/ios app if applicable.

### 0. Prerequisites

You will need to pip install `onnxruntime onnx onnxruntime_extensions transformers` as the necessary libraries.

```
    pip install onnx onnxruntime onnxruntime_extensions
```
```
    pip install transformers
```

To work with Python in Jupyter Notebooks, you must activate an [Anaconda](https://www.anaconda.com/) environment or another Python environment in which you've installed the [Jupyter package](https://pypi.org/project/jupyter/). 

In [2]:
import io
import numpy as np
import onnx
import onnxruntime as ort

###  1. Prepare ONNX Model from HuggingFace MobileBert model

In [3]:
import transformers
from transformers.onnx import FeaturesManager
from pathlib import Path
from onnxruntime.quantization import quantize_dynamic, QuantType

Original model: https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english

In [4]:
def create_onnx_model_from_huggingface(hf_model_name, onnx_model_path):
    """
        Load the model from huggingface and export it to onnx
    """
    tokenizer = transformers.AutoTokenizer.from_pretrained(hf_model_name)
    model = transformers.AutoModelForSequenceClassification.from_pretrained(hf_model_name)
    
    model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(model, feature="sequence-classification")
    onnx_config = model_onnx_config(model.config)
    
    # Export the hf model to onnx
    onnx_inputs, onnx_outputs = transformers.onnx.export(tokenizer, # pretrained generic tokenizer class for the model
                                                         model, # pretrained hf model
                                                         onnx_config, # onnx configurations which includes input/output names/types info
                                                         16, # opset_version - the ONNX version to export the model to
                                                         onnx_model_path) # where to save the exported onnx model

In [6]:
onnx_model_path = Path('mobilebert-uncased-mnli.onnx')
if not onnx_model_path.exists():
    print("Creating ONNX model from huggingface model...")
    create_onnx_model_from_huggingface('typeform/mobilebert-uncased-mnli', onnx_model_path)

Creating ONNX model from huggingface model...
================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



Check if the output ONNX model is exported successfully.

In [7]:
assert onnx_model_path.exists()

Quantize the output model.

In [8]:
def quantize_model(model_path: Path):
    """
        Quantize the model, so that it can be run on mobile devices with smaller memory footprint
    """
    quantized_model_path = model_path.with_name(model_path.stem+"_quant").with_suffix(model_path.suffix)
    quantize_dynamic(model_path, quantized_model_path, weight_type=QuantType.QInt8)
    model_path.unlink()
    return quantized_model_path

In [9]:
quantized_model = quantize_model(onnx_model_path)

Ignore MatMul due to non constant B: /[/mobilebert/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/mobilebert/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/mobilebert/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/mobilebert/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/mobilebert/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/mobilebert/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/mobilebert/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/mobilebert/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/mobilebert/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/mobilebert/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/mobilebert/encoder/layer.5/attention/self/MatMul]


###  2. Add pre and post processing steps to ONNX model

In [10]:
from onnxruntime_extensions.tools.pre_post_processing import *
from onnxruntime_extensions.tools import add_pre_post_processing_to_model as add_ppp
from contextlib import contextmanager

In [19]:
def add_pre_post_processing(input_model_path: Path, output_model_path: str, model_name: str = "typeform/mobilebert-uncased-mnli"):
    
    """
    Add pre and post processing to the model, for tokenization and post processing
    """
    onnx_opset = 16
    model = onnx.load(str(input_model_path.resolve(strict=True)))
    inputs = [create_named_value("input_text", onnx.TensorProto.STRING, [1, "num_sentences"])]  # Fix the batch size to be 1
    
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

    @contextmanager
    def temp_vocab_file():
        vocab_file = input_model_path.parent/ "vocab.txt"
        yield vocab_file

    with temp_vocab_file() as vocab_file:
        import json
        with open(str(vocab_file), 'w') as f:
            f.write(json.dumps(tokenizer.vocab))

        pipeline = PrePostProcessor(inputs, onnx_opset)
        
        tokenizer_args = TokenizerParam(
            vocab_or_file=vocab_file,
            do_lower_case=True,
        )
        
        pipeline.add_pre_processing(
            [
                BertTokenizer(tokenizer_args),
            ]
        )
        
        pipeline.add_post_processing(
            [
                ArgMax(),
            ]
        )

    new_model = pipeline.run(model)
    onnx.save_model(new_model, output_model_path)

In [20]:
output_model_path = str(quantized_model).replace(".onnx", "_with_pre_post_processing.onnx")
add_pre_post_processing(quantized_model, output_model_path)

### 3. Test output ONNX model

In [21]:
from onnxruntime_extensions import get_library_path

In [22]:
def test_onnx_model(model_path: str):
    
    so = ort.SessionOptions()
    so.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_EXTENDED

    # Note: register the custom operators for the image decode/encode pre/post processing provided by onnxruntime-extensions
    # with onnxruntime. if we do not do this we'll get an error on model load about the operators not being found.
    ortext_lib_path = get_library_path()
    so.register_custom_ops_library(ortext_lib_path)
    inference_session = ort.InferenceSession(model_path, so)
    
    text = 'Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app.';
    text_2 = 'I love transformers!'
    outputs = inference_session.run(['index'], {'input_text': [[text]]})
    output_answer = outputs[0]
    print("\nInput text: " + text)
    print("\nResult is: " + ("POSITIVE" if output_answer[0] == 1 else "NEGATIVE"))
    
    print("\nInput text: " + text_2)
    outputs = inference_session.run(['index'], {'input_text': [[text_2]]})
    output_answer = outputs[0]
    print("\nResult is: " + ("POSITIVE" if output_answer[0] == 1 else "NEGATIVE"))

In [23]:
test_onnx_model(output_model_path)


Input text: Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app.

Result is: POSITIVE

Input text: I love transformers!

Result is: POSITIVE


### 4. Build and run inference with the output model in a mobile application

You can plug in the generated onnx model and run inference easily in a mobile application (Android/iOS) with the pre/post processing support.

- **Android**

Example code snippet for initalizing ort session and register ort extensions for pre/post processing support:
```kotlin

   // Initialize Ort Session and register the onnxruntime extensions package that contains the custom operators.
    val sessionOptions: OrtSession.SessionOptions = OrtSession.SessionOptions()
    if (ep.contains("CPU")){
    } else if (ep.contains("NNAPI")) {
        sessionOptions.addNnapi()
    } else if (ep.contains("XNNAPCK")) {
        val po = mapOf<String, String>()
        sessionOptions.addXnnpack(po)
    }
    sessionOptions.setSessionLogLevel(OrtLoggingLevel.ORT_LOGGING_LEVEL_VERBOSE)
    sessionOptions.setSessionLogVerbosityLevel(0)
    sessionOptions.registerCustomOpLibrary(OrtxPackage.getLibraryPath())
    ortSession = ortEnv.createSession(readModel(), sessionOptions)
```

Example code snippet for performing Question Answering task with MobileBERT model in an Android app:
```kotlin

    // Step 1: Prepare the input text
    val text = text_seq.toString()

    // Step 2: Create input tensor
    val shape = longArrayOf(1, 1)
    val inputTensor = OnnxTensor.createTensor(ortEnv, arrayOf(text), shape)

    inputTensor.use {
        // Step 3: Call Ort InferenceSession Run
        val output = ortSession.run(Collections.singletonMap("input_text", inputTensor))

        // Step 4: Analyze the inference result
        output.use {
            val rawOutput = (output?.get(0)?.value) as Array<String>
            // POSITIVE if 1 else NEGATIVE
            outputAnswer = rawOutput[0]
        }
    }
```

- **iOS**

Example code snippet for performing Question Answering task in an iOS app:


```c
    // Step 1: Register custom ops
    const auto ort_log_level = ORT_LOGGING_LEVEL_INFO;
    auto ort_env = Ort::Env(ort_log_level, "ORTObjectDetection");
    auto session_options = Ort::SessionOptions(); 
    
    if (RegisterCustomOps(session_options, OrtGetApiBase()) != nullptr) {
        throw std::runtime_error("RegisterCustomOps failed");
    }
            
    // Step 2: Load model   
    NSString *model_path = [NSBundle.mainBundle pathForResource:@"yolov8n_with_pre_post_processing"
                                                                ofType:@"onnx"];
    if (model_path == nullptr) {
        throw std::runtime_error("Failed to get model path");
    }
            
    // Step 2: Create Ort Inference Session    
    auto sess = Ort::Session(ort_env, [model_path UTF8String], session_options);
            
    // Step 3: Prepare input tensors and input/output names        
    std::vector<int64_t> input_dims{1, 1};
    Ort::AllocatorWithDefaultOptions ortAllocator;
    auto input_tensor = Ort::Value::CreateTensor(ortAllocator, input_dims.data(), input_dims.size(), ONNX_TENSOR_ELEMENT_DATA_TYPE_STRING);
P
    std::vector<std::string> input_vec;
    input_vec.reserve(1);
    input_vec.push_back([input_user_text UTF8String]);
    std::vector<const char*> p_str;
    for (const auto& s : input_vec) {
        p_str.push_back(s.c_str());
    }
    input_tensor.FillStringTensor(p_str.data(), p_str.size());
 
    // Step 4: Call inference session run
    constexpr auto input_names = std::array{"input_text"};
    constexpr auto output_names = std::array{"index"};
    const auto outputs = sess.Run(Ort::RunOptions(), input_names.data(),
                                &input_tensor, 1, output_names.data(), 1);
                    
    // Step 5: Analyze model outputs
    if (outputs.size() != 1) {
        throw std::runtime_error("Unexpected number of outputs");
    }        
    const auto &output_tensor = outputs.at(1);
    const std::string* output_string_raw = output_tensor.GetTensorData<int64_t>();
```

**Note**: Model input/output names, sizes, types may require corresponding adjustment according to specific models using. The above act as sample code for demonstration usage purpose.